In [ ]:
from data.config import data_config
import pandas as pd
import numpy as np
from datamanager import DataManager
# import shap
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error as mse, make_scorer
import torch
import torch.nn as nn


pd.options.display.float_format = '{:.2f}'.format
# shap.initjs()

In [ ]:
dm_w_lags = DataManager(data_config=data_config, target_col="stator_winding")
dm_w_lags.featurize()
X_train, y_train, X_val, y_val, X_test, y_test, _ = dm_w_lags.split_data(scale=True)


In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

param_grid = {
    'hidden_layer_sizes': (128, 64, 32),
    'activation': 'relu',
    'solver': 'adam',
    'alpha': 0.001,
    'learning_rate': 'adaptive',
}

# Create an instance of MLPRegressor
mlp = MLPRegressor(max_iter=1000, early_stopping=True, random_state=42, batch_size='auto', verbose=True, **param_grid)
mlp.fit(X_train, np.squeeze(y_train))



In [ ]:
y_pred = mlp.predict(X_test)

# Calculate evaluation metrics
# best pm yet - 4.65                           5.46 - 'hidden_layer_sizes': (256, 128, 64),
# best stator_yoke - 3.68 - (128, 64, 32)
# best stator tooth - 
mse = mean_squared_error(y_test, y_pred)
mse

In [ ]:
import matplotlib.pyplot as plt
plt.plot(mlp.loss_curve_)
plt.xlabel("epochs")
plt.ylabel("MSE score")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(len(y_test)), y_test, label="true")
plt.plot(y_pred, label="pred")
plt.legend()
plt.show()